In [5]:
import json

In [6]:
train_dict = json.load(open('../data/coqa-train-v1.0.json', encoding='utf8'))
dev_dict = json.load(open('../data/coqa-dev-v1.0.json', encoding='utf8'))

In [53]:
dev_dict['data'][0]['additional_answers']['0'][2]

{'span_start': 192,
 'span_end': 215,
 'span_text': "But Cotton wasn't alone",
 'input_text': 'no',
 'turn_id': 3}

In [78]:
qa_range = 5
def get_text_list(data):
    return_list = []
    for item in data:
        for cutoff in range(1, len(item['questions'])):
            text = ''
            text += 'Story:\n' + item['story'] + '\n\n'
            text += 'Discussion:\n'
            text += '\n'.join(['Q: ' + q['input_text'] 
                               + '\nA: ' + a['input_text'] 
                               for q, a in zip(item['questions'][max(0, cutoff - qa_range):cutoff], 
                                               item['answers'][max(0, cutoff - qa_range):cutoff])
                              ])
            answer = item['answers'][cutoff - 1]['input_text']
            text = '\n'.join(text.split('\n')[:-1])
            return_list.append({'inputs': text, 'labels': answer})

    return return_list

def get_text_list_for_dev(data):
    return_list = []
    for item in data:
        for cutoff in range(1, len(item['questions'])):
            text = ''
            text += 'Story:\n' + item['story'] + '\n\n'
            text += 'Discussion:\n'
            text += '\n'.join(['Q: ' + q['input_text'] 
                               + '\nA: ' + a['input_text'] 
                               for q, a in zip(item['questions'][max(0, cutoff - qa_range):cutoff], 
                                               item['answers'][max(0, cutoff - qa_range):cutoff])
                              ])
            answers = [item['answers'][cutoff - 1]['input_text']]
        
            for key in item['additional_answers']:
                answers.append(item['additional_answers'][key][cutoff - 1]['input_text'])
            text = '\n'.join(text.split('\n')[:-1])
            return_list.append({'inputs': text, 'labels': list(set(answers))})

    return return_list

In [79]:
train_texts_list = get_text_list(train_dict['data'])
dev_texts_list = get_text_list_for_dev(dev_dict['data'])

In [80]:
train_texts_list[0]

{'inputs': 'Story:\nThe Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at

In [84]:
dev_texts_list[3]

{'inputs': 'Story:\nOnce upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer\'s horses slept. But Cotton wasn\'t alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton\'s mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer\'s orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. \n\n"What are you doing, Cotton?!" \n\n"I only wanted to be more like you". \n\nCotton\'s mommy rubbed her face on Cotton\'s and said "Oh Cotton, but your fur is so pretty and special, like you. We 

In [85]:
with open('../data/qa_train_BART.json', 'w') as f:
    json.dump(train_texts_list, f)
with open('../data/qa_dev_BART.json', 'w') as f:
    json.dump(dev_texts_list, f)